## Projeto Final

In [26]:
import csv
from datetime import datetime

registros = []

def ler_relatorio():
    relatorio_anterior = "relatorio_registros.csv"
    try:
        with open(relatorio_anterior, mode='r', newline='') as arquivo:
            leitor_arquivo = csv.DictReader(arquivo)

            for linha in leitor_arquivo:
                registros.append(linha)
        print("Registros carregados com sucesso!")

    except FileNotFoundError:
        print(f"\nAinda não existe um arquivo de relatório. Crie um novo arquivo.")


def obter_id_usuario():
    while True:
        id_usuario = input("Digite seu ID: ")
        if id_usuario not in registros:
            return id_usuario
        else:
            print("Esse ID já existe. Por favor, insira outro.")


def obter_tipo_movimentacao():
    while True:
        tipo_registro = input("\n-----------------------------------------\nQual movimentação deseja fazer? Digite \n'1' para 'Receita' \n'2' para 'Despesa' \n'3' para 'Investimento': ")
        if tipo_registro.isdigit() and int(tipo_registro) in [1, 2, 3]:
            return int(tipo_registro)
        else:
            print("Por favor, digite uma opção válida (de 1 a 3).")


def obter_valor():
    while True:
        valor_input = input("\nDigite um valor: ")
        if valor_input.replace('.', '', 1).isdigit():
            return float(valor_input)
        else:
            print("Por favor, digite um valor numérico.")


def criar_registro(id, valor, tipo_registro):
    data_str = input("Digite a data de hoje (DD/MM/YYYY):") 
    data_obj = datetime.strptime(data_str, "%d/%m/%Y")
    data = data_obj.strftime("%d/%m/%Y")

    if tipo_registro == 1:
        descricao_tipo = 'Receita'
    elif tipo_registro == 2:
        descricao_tipo = 'Despesa'
        valor = -abs(valor)  # Despesas com valores negativos
    elif tipo_registro == 3:
        descricao_tipo = 'Investimento'
        taxa_juros_anual = float(input("Digite a taxa de juros anual: "))
        taxa_juros_diaria = (1 + (taxa_juros_anual/100)) ** (1/360) - 1

        data_vencimento_str = input("Digite a data de vencimento do seu investimento (DD/MM/YYYY): ")
        data_vencimento = datetime.strptime(data_vencimento_str, "%d/%m/%Y")

        dias = (data_vencimento - data_obj).days
        valor = valor * (1 + taxa_juros_diaria) ** dias # montante
    else:
        print("Tipo de movimentação inválido.")
        return

    registro = {'id': id, 'valor': round(valor, 2), 'tipo_registro': tipo_registro, 'descricao_tipo': descricao_tipo, 'data': data}
    registros.append(registro)
    print(f"\nRegistro criado para o usuário {id}: Tipo {descricao_tipo}, Valor {round(valor, 2)}, Data {data}")


def ler_registro_por_data(data_registro):
    is_date = lambda registro: registro["data"] == data_registro
    return list(filter(is_date, registros))


def ler_registro_por_tipo(tipo_registro):
    is_tipo = lambda registro: registro["tipo_registro"] == str(tipo_registro)
    return list(filter(is_tipo, registros))


def ler_registro_por_valor(valor_registro):
    is_valor = lambda registro: registro["valor"] == valor_registro
    return list(filter(is_valor, registros))


def ler_registros(opcao):
    if opcao == '1':
      busca_data = input("Qual data deseja buscar?")
      return ler_registro_por_data(busca_data)

    elif opcao == '2':
      busca_tipo = input("Qual tipo deseja buscar?")
      return ler_registro_por_tipo(busca_tipo)

    elif opcao == '3':
      busca_valor = input("Qual valor deseja buscar?")
      return ler_registro_por_valor(busca_valor)

    elif opcao == '4':
      return "Busca por leitura de registros encerrada."

    else:
      return f"Opção escolhida: {opcao}. Por favor, digite uma opção válida (de 1 a 4)."


def atualizar_registro(id, novo_valor, novo_tipo):
    for registro in registros:
        if registro['id'] == id:
            registro['tipo_registro'] = novo_tipo
            if novo_tipo == 2:
                registro['valor'] = -abs(novo_valor)
            else:
                registro['valor'] = novo_valor
            registro['data'] = datetime.now().strftime("%d/%m/%Y")
            print("\nRegistro atualizado com sucesso!")
            return
    print("ID não encontrado. Nenhum registro atualizado.")


def deletar_registro(registros, id):
    for registro in registros:
        if registro['id'] == id:
            registros.remove(registro)
            print(f"\nRegistro com ID {id} deletado com sucesso!")
            return registros


def exportar_relatorio(registros):
    try:
      with open("relatorio_registros.csv", mode='w', newline='') as arquivo_csv:
          colunas = ['id', 'valor', 'tipo_registro', 'descricao_tipo', 'data']
          escritor_arquivo = csv.DictWriter(arquivo_csv, fieldnames=colunas)
          escritor_arquivo.writeheader()
          escritor_arquivo.writerows(registros)
    except FileNotFoundError as e:
        print("\nErro ao exportar relatório: ", e)
    print('\nRelatório exportado com sucesso!')


# def agrupamento():
#   precisa criar a função de agrupamento para mostrar o total de valor baseado em alguma informação (mês, tipo...


ler_relatorio()


continuar = True

while continuar:
    operacao = input("\nQual operação deseja realizar? Digite \n'1' para 'Criar Registro' \n'2' para 'Ler Registro' \n'3' para 'Atualizar Registro' \n'4' para 'Deletar Registro' \n'5' para 'Finalizar': \n-----------------------------------------\n")

    if operacao == '1':
        print('Operação escolhida: Criar Registro\n-----------------------------------------\n')
        id = obter_id_usuario()
        tipo_registro = obter_tipo_movimentacao()
        valor = obter_valor()
        criar_registro(id, valor, tipo_registro)

    elif operacao == '2':
        opcao_busca = input("\nComo deseja realizar a busca do registro? Digite \n'1' para 'Buscar por Data' \n'2' para 'Buscar por Tipo' \n'3' para 'Buscar por Valor': ")
        ler_registros(opcao_busca)

    elif operacao == '3':
        id = input("\nDigite o ID do registro que deseja atualizar: ")
        novo_valor = obter_valor()
        novo_tipo = obter_tipo_movimentacao()
        atualizar_registro(id, novo_valor, novo_tipo)

    elif operacao == '4':
        id = input("\nDigite o ID do registro que deseja deletar: ")
        deletar_registro(id)

    elif operacao == '5':
        exportar_relatorio(registros)
        print("Operação encerrada.")
        continuar = False

    else:
        print(f"Opção escolhida: {operacao}. Por favor, digite uma opção válida (de 1 a 5).")



Registros carregados com sucesso!


Operação escolhida: Criar Registro
-----------------------------------------

data: 2024-01-25 00:00:00
data de vencimento: 2024-06-19 00:00:00

Registro criado para o usuário 8: Tipo Investimento, Valor 1039.41, Data 25/01/2024

Relatório exportado com sucesso!
Operação encerrada.
